# English to Bengali translation using RNN

Status: `In-Development`

Dataset URL: http://www.manythings.org/anki


In [86]:
# IMPORT LIBRARIES
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Data Processing

In [1]:
# DOWNLOAD DATASET
!!curl -O http://www.manythings.org/anki/ben-eng.zip
!!unzip ben-eng.zip

['Archive:  ben-eng.zip',
 '  inflating: _about.txt              ',
 '  inflating: ben.txt                 ']

In [100]:
# CONFIG FILE
batch_size = 64         # Batch size for training.
epochs = 100            # Number of epochs to train for.
latent_dim = 256        # Latent dimensionality of the encoding space.
num_samples = 10000     # Number of samples to train on.
RNN_units = 512         # Intermediate RNN units
data_path = "ben.txt"   # Path to the data txt file on disk.

In [5]:
# OPEN THE DATA FILE
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

# SHOWING FIRST 10 LINES
lines[:10]

['Go.\tযাও।\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5545004 (tanay)',
 'Go.\tযান।\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5545005 (tanay)',
 'Go.\tযা।\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5545006 (tanay)',
 'Run!\tপালাও!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #5548781 (tanay)',
 'Run!\tপালান!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #5548783 (tanay)',
 'Who?\tকে?\tCC-BY 2.0 (France) Attribution: tatoeba.org #2083030 (CK) & #5548787 (tanay)',
 'Wow!\tবাহ!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #11026314 (robingaspi)',
 'Fire!\tআগুন!\tCC-BY 2.0 (France) Attribution: tatoeba.org #1829639 (Spamster) & #3232240 (tanay)',
 'Help!\tবাঁচাও!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #5548780 (tanay)',
 'Help!\tবাঁচান!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #5548782 (tanay)']

In [45]:
print(f"Total number of lines: {len(lines)}")

Total number of lines: 5937


In [26]:
# SEPARATING DATA FROM LINES
def to_lines(text):
    sentence = text.strip().split("\n")
    sentence_list = [a.split("\t") for a in sentence]
    return sentence_list[0]

# TEST
to_lines(lines[0])

['Go.',
 'যাও।',
 'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5545004 (tanay)']

In [54]:
# SEPARATE BEN-ENG COLUMNS FROM THE LIST
eng_ben = np.array([to_lines(a) for a in lines][:5936])
eng_ben = eng_ben[:,[0,1]]

# TEST
eng_ben[:5]

array([['Go.', 'যাও।'],
       ['Go.', 'যান।'],
       ['Go.', 'যা।'],
       ['Run!', 'পালাও!'],
       ['Run!', 'পালান!']], dtype='<U134')

In [59]:
# CASE CONVERSION
for i in range(len(eng_ben)):
    eng_ben[i,0] = eng_ben[i,0].lower()

# TEST
eng_ben[:5]

array([['go.', 'যাও।'],
       ['go.', 'যান।'],
       ['go.', 'যা।'],
       ['run!', 'পালাও!'],
       ['run!', 'পালান!']], dtype='<U134')

## Vocabulary building

In [60]:
# WORD TOKENIZATION
def tokenization(lines):
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [92]:
# TOKENIZE ENGLISH SENTENCES
eng_tokenizer = tokenization(eng_ben[:,0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
print(f"length of english vocabulary: {eng_vocab_size}")

length of english vocabulary: 2516


In [93]:
# TOKENIZE BENGALI SENTENCES
ben_tokenizer = tokenization(eng_ben[:,1])
ben_vocab_size = len(ben_tokenizer.word_index) + 1
print(f"length of bengali vocabulary: {ben_vocab_size}")

length of bengali vocabulary: 4489


In [82]:
# DETERMINE THE MAXLENGTH
eng_max_len = -1
ben_max_len = -1
for i in eng_ben:
    temp_eng = len(str(i[0]).strip().split(" "))
    temp_ben = len(str(i[1]).strip().split(" "))
    if eng_max_len < temp_eng:
        eng_max_len = temp_eng
    if ben_max_len < temp_ben:
        ben_max_len = temp_ben

# PRINTING THE MAX LENGTH
print(f"English max length: {eng_max_len}\nBengali max length: {ben_max_len}")

English max length: 20
Bengali max length: 18


In [95]:
# ENCODE SEQUENCES
def encode_sequences(tokenizer, length, lines):
    seq = tokenizer.texts_to_sequences(lines)       # INTEGER ENCODING
    seq = tf.keras.utils.pad_sequences(seq, maxlen = length, padding = 'post')
    return seq

In [96]:
# SPLIT INTO TRAIN AND TEST
train, test = train_test_split(eng_ben, test_size = 0.1, random_state = 12)

In [97]:
# PREPARING TRAINING SET
trainX = encode_sequences(eng_tokenizer, eng_max_len, train[:, 0])
trainY = encode_sequences(ben_tokenizer, ben_max_len, train[:, 1])

In [98]:
# PREPARING VALIDATION SET
testX = encode_sequences(eng_tokenizer, eng_max_len, test[:, 0])
testY = encode_sequences(ben_tokenizer, ben_max_len, test[:, 1])

## Model

In [102]:
# IMPORT
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras import optimizers

In [99]:
# MODEL DECLARATION
def define_model(in_vocab, out_vocab, in_timestamp, out_timestamp, units):
    model = Sequential()
    model.add(Embedding(in_vocab, units, input_length = in_timestamp, mask_zero = True))
    model.add(LSTM(units))
    model.add(RepeatVector(out_timestamp))
    model.add(LSTM(units, return_sequences = True))
    model.add(Dense(out_vocab, activation = "softmax"))
    return model

In [105]:
# MODEL DEFINITION
model = define_model(eng_vocab_size, ben_vocab_size, eng_max_len, ben_max_len, RNN_units)

# COMPILATION
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.001), loss = "sparse_categorical_crossentropy")

# SUMMARY
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 512)           1288192   
                                                                 
 lstm_4 (LSTM)               (None, 512)               2099200   
                                                                 
 repeat_vector_2 (RepeatVec  (None, 18, 512)           0         
 tor)                                                            
                                                                 
 lstm_5 (LSTM)               (None, 18, 512)           2099200   
                                                                 
 dense_2 (Dense)             (None, 18, 4489)          2302857   
                                                                 
Total params: 7789449 (29.71 MB)
Trainable params: 7789449 (29.71 MB)
Non-trainable params: 0 (0.00 Byte)
______________

In [107]:
# TRAINING
model.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1),
          epochs = 10, batch_size = 512, validation_split = 0.2)

Epoch 1/10
9/9 [==============================] - 78s 9s/step - loss: 2.3386 - val_loss: 2.2326
Epoch 2/10
9/9 [==============================] - 83s 9s/step - loss: 2.2543 - val_loss: 2.1954
Epoch 3/10
9/9 [==============================] - 79s 9s/step - loss: 2.1822 - val_loss: 2.1446
Epoch 4/10
9/9 [==============================] - 92s 10s/step - loss: 2.1429 - val_loss: 2.0463
Epoch 5/10
9/9 [==============================] - 103s 11s/step - loss: 2.0912 - val_loss: 1.9996
Epoch 6/10
9/9 [==============================] - 94s 10s/step - loss: 2.0072 - val_loss: 2.0233
Epoch 7/10
9/9 [==============================] - 81s 9s/step - loss: 2.0427 - val_loss: 2.0083
Epoch 8/10
9/9 [==============================] - 96s 11s/step - loss: 1.9864 - val_loss: 1.9149
Epoch 9/10
9/9 [==============================] - 96s 11s/step - loss: 1.9761 - val_loss: 1.9079
Epoch 10/10
9/9 [==============================] - 99s 11s/step - loss: 1.9329 - val_loss: 1.8928


## Prediction

In [ ]:
# PREDICTION
predictions = np.argmax(model.predict(testX.reshape((testX.shape[0], testX.shape[1]))), axis = 1)

In [150]:
predictions

array([[17,  1,  1, ...,  0,  0,  0],
       [17,  1,  1, ...,  0,  0,  0],
       [17,  1,  1, ...,  0,  0,  0],
       ...,
       [17,  1,  1, ...,  0,  0,  0],
       [17,  1,  1, ...,  0,  0,  0],
       [17,  1,  1, ...,  0,  0,  0]])

In [151]:
# GET WORD FROM THE PREDICTIONS
def get_word(n, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == n:
            return word
    return None

In [163]:
# CONVERT PREDICTIONS INTO SENTENCES
pred_text = []
for i in predictions:
    temp = []
    for j in range(len(i)):
        t = get_word(i[j], ben_tokenizer)
        if j > 0:
            if t == get_word(i[j-1], eng_tokenizer) or (t == None):
                pass
            else:
                temp.append(t)
        else:
            if t == None:
                temp.append("")
            else:
                temp.append(t)
    pred_text.append(" ".join(temp))
    break

In [161]:
print(test[:,1][0])

আপনার হাতে সময় আছে?


In [164]:
print(pred_text)

[0, 'টমের আমি আমি আমি আমি আমার আমি আমি আমি আমি টম টম টম টম টম আমি আমি আমি টম টম আমি টম টম টম টম টম টম টম টম টম টম টম টম টম আমি টম টম টম টম আমি টম টম টম টম টম টম আমি টম টম টম টম টম টম টম টম টম টম টম টম টম টম আমার টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম আমার টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম আমি টম টম টম টম টম টম আমি টম আমি টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম আমি টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম টম আমি টম টম টম টম আমি আমি টম টম আমি টম টম টম টম টম আমি টম টম টম টম টম টম আমি টম টম টম টম টম টম টম টম আমি টম টম টম টম টম টম আমি টম টম টম টম টম টম আমি টম টম টম আমি টম টম টম আমি আমি আমি টম আমি টম আমি টম টম টম টম টম টম টম টম টম আমি টম টম আমি টম টম টম আমি টম টম টম টম টম টম টম টম টম টম টম টম আমি আমি আমি টম টম টম টম টম টম টম টম টম টম টম টম টম আমি টম আমি টম টম টম আমি টম আমি টম টম টম টম আমি টম আমি আমি আমি টম টম টম টম টম আমি টম আমি টম টম টম আমি টম টম টম আমি

## Test

In [ ]:
# CREATE DATA FRAME WITH ORIGINAL VS PREDICTED
df = pd.DataFrame({"actual": test[:,0], "generated": pred_text})
df.head(10)

```
References: https://youtu.be/7gHqxK1o7MU
```